# Error Analysis

Time steps error

In [ ]:
import numpy as np
import math
from scipy.stats import norm

Options parameters

In [ ]:
S_0 = 100  # asset price at t=0 
T = 1 # time to maturity in years
K = 50 # strike price
r = 0.05 # risk-free interest rate
sigma = 0.6 # volatility
k = 10 # time steps
M = 100000 # samples
dt = T/k # time step

Black-Scholes formula

In [ ]:
d_1 = (np.log(S_0/K)+(r+0.5*sigma**2)*T)/(sigma*T**0.5)
d_2 = (np.log(S_0/K)+(r-0.5*sigma**2)*T)/(sigma*T**0.5)

C_0_true = S_0*norm.cdf(d_1)-math.exp(-r*T)*K*norm.cdf(d_2)
print(f'European Call (BS): {C_0_true}')

Run Monte Carlo method

In [ ]:
lnS = np.log(S_0)

sum_CT = 0
sum_CT2 = 0

for i in range(M):
    lnSt = lnS

    for j in range(k):
        lnSt = lnSt + (r - 0.5*sigma**2)*dt + sigma*np.sqrt(dt)*np.random.normal()
    
    ST = np.exp(lnSt)
    CT = max(0, ST-K)

    sum_CT += CT
    sum_CT2 += CT**2

C0 = np.exp(-r*T)*sum_CT/M
std = np.sqrt((sum_CT2-sum_CT*sum_CT/M)*np.exp(-2*r*T) / (M-1) )
SE = std/np.sqrt(M)

print(C0)

Run experiments with different number of time steps

In [ ]:
lnS = np.log(S_0)
max_iterations = 1
max_samples = 23
M_list = [2**(i+1) for i in range(max_samples)]
k_list = [1, 5, 10] # time steps

for k in k_list:
    dt = T/k
    results = []
    for experiment in range(max_iterations):
        
        estimations = []
        standard_deviations = []
        errors = []

        for M in M_list:
            
            # generate sample
            Z = np.random.normal(size=(k, M)) 
            delta_lnSt = (r - 0.5*sigma**2)*dt + sigma*np.sqrt(dt)*Z
            lnSt = lnS + np.cumsum(delta_lnSt, axis=0)
            lnSt = np.concatenate((np.full(shape=(1, M), fill_value=lnS), lnSt))
            
            # compute Expectation and SE
            ST = np.exp(lnSt)
            CT = np.maximum(0, ST - K)
            C0 = np.exp(-r*T)*np.sum(CT[-1])/M
            std = np.sqrt( np.sum( (CT[-1] - C0)**2) / (M-1) )
            SE = std/np.sqrt(M)
            estimations.append(C0)
            standard_deviations.append(std)
            errors.append(abs(C0-C_0_true)/C_0_true)
       
        results.append([estimations, standard_deviations, errors, M_list])
    
    print("Call value is ${0} with SE +/- {1}".format(np.round(C0,2),np.round(SE,2)))
    